Dummy

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


import sys

sys.path.append('../src')

from data_preprocessing import X, y

In [10]:
# Definiera cv (X och y importeras redan från data_preprocessing)
import sklearn.model_selection as skl_ms

cv = skl_ms.StratifiedKFold(n_splits=5, random_state=1, shuffle=True)


In [11]:
day_cols = [c for c in X.columns if c.startswith("day_")]
binary_cols = ["snow_or_not", "summertime"]
cols_to_scale = [c for c in X.columns if c not in day_cols + binary_cols]


preprocessor = ColumnTransformer(
    transformers=[("scale", StandardScaler(), cols_to_scale)],
    remainder="passthrough",
)


dummy_pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("model", DummyClassifier(strategy='stratified', random_state=1))
])


baseline_score = cross_val_score(dummy_pipeline, X, y, cv=cv, scoring='accuracy').mean()

print(f"Baseline (Dummy) Accuracy: {baseline_score:.4f}")

Baseline (Dummy) Accuracy: 0.6844


In [27]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

strategies = ['stratified', 'most_frequent', 'uniform']
for strategy in strategies:
    dummy = DummyClassifier(strategy=strategy, random_state=1)
    pipeline = Pipeline([("preprocess", preprocessor), ("model", dummy)])

    y_pred = cross_val_predict(pipeline, X, y, cv=cv)

    print(f"\n{strategy}")
    print(classification_report(y, y_pred, zero_division=0))


stratified
                  precision    recall  f1-score   support

high_bike_demand       0.18      0.20      0.19       238
 low_bike_demand       0.81      0.80      0.80      1042

        accuracy                           0.68      1280
       macro avg       0.50      0.50      0.50      1280
    weighted avg       0.70      0.68      0.69      1280


most_frequent
                  precision    recall  f1-score   support

high_bike_demand       0.00      0.00      0.00       238
 low_bike_demand       0.81      1.00      0.90      1042

        accuracy                           0.81      1280
       macro avg       0.41      0.50      0.45      1280
    weighted avg       0.66      0.81      0.73      1280


uniform
                  precision    recall  f1-score   support

high_bike_demand       0.17      0.44      0.25       238
 low_bike_demand       0.80      0.52      0.63      1042

        accuracy                           0.50      1280
       macro avg       0.49 